# EEG resting state ML classification project 

In [ ]:
# load basic libraries
import pandas as pd
import numpy as np
import mne
from pathlib import Path
import pickle
import time

from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV, cross_val_score

%matplotlib widget
import matplotlib
import matplotlib.pyplot as plt

# set directories
# %cd D:/Programy/Anaconda3/Projects/EEG ML project # working directory
%cd D:
pkls = './Pickles/' # objects & variables

In [ ]:
# to see all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Load already prepared data

In [ ]:
# load time-series data, non-averaged
with open(pkls +'data_srf.pkl', 'rb') as handle:
    srf = pickle.load(handle)
srf.shape

In [ ]:
# load time-series data, non-averaged - open & closed eyes (oc) sessions
with open(pkls +'data_srf_oc.pkl', 'rb') as handle:
    srf_oc = pickle.load(handle)
srf_oc.shape

In [ ]:
# load time-series data, non-averaged - video watching (vw) sessions
with open(pkls +'data_srf_vw.pkl', 'rb') as handle:
    srf_vw = pickle.load(handle)
srf_vw.shape

In [ ]:
# load averaged data with eeg only
with open(pkls +'data_ave.pkl', 'rb') as handle:
    ave = pickle.load(handle)
ave.shape

In [ ]:
# load averaged data with eeg and questionnaire
with open(pkls +'data_aveq.pkl', 'rb') as handle:
    aveq = pickle.load(handle)
aveq.shape

In [ ]:
# load split sets, eeg only
with open(pkls +'xy_train.pkl', 'rb') as handle:
    x_train = pickle.load(handle)
    y_train = pickle.load(handle)
y_train.shape
x_train.shape

In [ ]:
# load split sets, eeg and questionnaire
with open(pkls +'xy_train_q.pkl', 'rb') as handle:
    x_train = pickle.load(handle)
    y_train = pickle.load(handle)
y_train.shape
x_train.shape

In [ ]:
# load x train reduced (post-PCA), eeg only
with open(pkls +'x_train_reduced.pkl', 'rb') as handle:
    x_train_reduced = pickle.load(handle)

# Prepare data from scratch

In [ ]:
# local computer
%cd D:
path_oc = Path('D:/Programy/Anaconda3/Projects/EEG ML project/Epochs clean/)
path_vw = Path('D:/Programy/Anaconda3/Projects/EEG ML project/Epochs clean/video')
epo_list_oc = list(path_oc.glob('*.fif'))
epo_list_vw = list(path_vw.glob('*.fif'))
len(epo_list_oc)
len(epo_list_vw)

In [ ]:
# remote computer
%cd D:
path_oc = Path('D:/Marcin/OneDrive - Uniwersytet Jagielloński/Nauka/Projekty/2020.02 - Bartek ML EEG/Epochs clean/')
path_vw = Path('D:/Marcin/OneDrive - Uniwersytet Jagielloński/Nauka/Projekty/2020.02 - Bartek ML EEG/Epochs clean/video')
epo_list_oc = list(path_oc.glob('*.fif'))
epo_list_vw = list(path_vw.glob('*.fif'))
len(epo_list_oc)
len(epo_list_vw)

#### Load and wrangle 

In [ ]:
# load open and closed eyes sessions
srf_oc = pd.DataFrame();
for file in epo_list_oc:
    rf = mne.read_epochs(fname = file).to_data_frame();
    rf['id'] = file.stem[:5];
    rf.drop('Status', axis = 1, inplace = True)
    display(rf.shape);
    srf_oc = srf_oc.append(rf);

In [ ]:
# load video watching sessions
srf_vw = pd.DataFrame();
for file in epo_list_vw:
    rf = mne.read_epochs(fname = file).to_data_frame();
    rf['id'] = file.stem[6:11];
    rf.drop('Status', axis = 1, inplace = True)
    display(rf.shape);
    srf_vw = srf_vw.append(rf);

In [ ]:
# preview
display(srf_oc)
srf_oc.shape

display(srf_vw)
srf_vw.shape

In [ ]:
# dump to pickle - oc
with open(pkls + 'data_srf_oc.pkl', 'wb') as handle:
    pickle.dump(srf_oc, handle, protocol=4)

In [ ]:
# dump to pickle - vw
with open(pkls + 'data_srf_vw.pkl', 'wb') as handle:
    pickle.dump(srf_vw, handle, protocol=4)

In [ ]:
# merge sessions
srf = srf_oc.append(srf_vw)

In [ ]:
# dump to pickle - merged
with open(pkls + 'data_srf.pkl', 'wb') as handle:
    pickle.dump(srf, handle, protocol=4)

In [ ]:
# group and re-index
avep = srf.groupby(['condition', 'epoch', 'id']).mean() # group means

avep = avep.rename_axis(columns = 'indx') # reset index
avep.reset_index(inplace=True)

avep.sort_values(by = ['id', 'epoch'], inplace=True) # sort and reindex
avep.reset_index(inplace=True, drop=True)

avep

In [ ]:
# count epochs per condition and id
avep.groupby(['id', 'condition']).agg('count')['epoch']

# mutate values
avepc = avep.replace({'condition': {'CLOSED/LONG': 0, 'CLOSED/SHORT': 0, # CE
                                    'OPEN/LONG': 1, 'OPEN/SHORT': 1,     # OE
                                    'VIDEO': 2 }})                       # VW
avepc.groupby(['condition']).agg('count')['epoch']

In [ ]:
avepc.isnull().values.any()
avepc

## Add questionnaire data 

In [ ]:
# load
arsq = pd.read_csv('./ARSQ2/eyes_opened_closed_factors.tsv', sep = '\t')
arsq.columns.values[0] = 'id'

# merge
avepcq = pd.merge(avepc, arsq, how = 'left', on = 'id') # left join on id

In [ ]:
# get ids with no ARSQ data
g = avepcq.groupby('id')['Discontinuity of mind']
g = g.count().rsub(g.size(), axis = 0)
g[g != 0]

In [ ]:
# imputer for ARSQ missing values
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy = 'median') # apply only to numeric columns

apq = avepcq.loc[:, 'Discontinuity of mind':'Verbal thought']
imputer.fit(apq) # stored in imputer.statistics_ (same as ap_num.median().values)

ap_imp = imputer.transform(apq)
ap_imp = pd.DataFrame(ap_imp, columns = apq.columns, index = apq.index) # add transformed features to Pandas df
ap_imp
ap_imp.shape

avepcq = pd.merge(avepc, ap_imp, how = 'left', left_index=True, right_index=True) # left join on id
avepcq

In [ ]:
# check numbers
avepcq.drop(['epoch', 'condition'], axis = 1).groupby(['id']).first()
avepcq.drop(['epoch', 'condition'], axis = 1).groupby(['id']).mean()

# avepcq.iloc[:, np.r_[2, 77:87]].groupby('id').median() # get median by id
# avepcq.iloc[:, np.r_[2, 77:87]].iloc[1:10].median() # check grand medians

## Add markers features 

In [ ]:
# load
with open(pkls +'features_r3.pkl', 'rb') as handle:
    ftrs = pickle.load(handle)
    
# count epochs per participant in each df
# avepc.groupby(['id']).agg('count')['epoch']
# ftrs.groupby(['id']).agg('count')['epoch'] 

ftrs
ftrs.shape

In [ ]:
 # no questionnaire 
ave = pd.merge(avepc, ftrs, how = 'left', on = ['id', 'epoch'])
ave = ave.drop(ave.loc[:, 'EXG1': 'HEOG'].columns, axis = 1) # drop EXT electrodes
ave.shape
# list(ave.columns.values) # view all columns names

In [ ]:
# with questionnaire
aveq = pd.merge(avepcq, ftrs, how = 'left', on = ['id', 'epoch'])
aveq = aveq.drop(aveq.loc[:, 'EXG1': 'HEOG'].columns, axis = 1) # drop EXT electrodes
aveq.shape
# list(aveq.columns.values) # view all columns names

In [ ]:
ave.info()
# ave.describe()
avepc.groupby(['condition']).agg('count')['epoch']

In [ ]:
# save full data, no questionnaire
with open(pkls + 'data_ave.pkl', 'wb') as handle:
    pickle.dump(ave, handle)

In [ ]:
# save full data, with questionnaire
with open(pkls + 'data_aveq.pkl', 'wb') as handle:
    pickle.dump(aveq, handle)

# Create a test set

In [ ]:
import sklearn
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit

# ave = aveq
ave.shape

In [ ]:
# stratified split
split = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
for train_index, test_index in split.split(ave, ave['condition']):
    strat_train_set = ave.loc[train_index]
    strat_test_set = ave.loc[test_index]

In [ ]:
# check ratios after splitting
ave['condition'].value_counts() / len(ave) # all data
strat_train_set['condition'].value_counts() / len(strat_train_set)
strat_test_set['condition'].value_counts() / len(strat_test_set)

In [ ]:
# copy train set, set aside test set
ap = strat_train_set.copy().sort_index()
ap

In [ ]:
# divide X (features) and y (target)
x_train_pre = ap.drop(['condition', 'id', 'epoch'], axis = 1) 
y_train = ap[['condition']]

# encode target variable
# from sklearn.preprocessing import LabelEncoder
# encoder = LabelEncoder()
# y_train_cat = y_train.apply(encoder.fit_transform).rename(columns={'condition':'condcat'})
# y_train_labs = pd.concat([y_train, y_train_cat], axis = 1)
# y_train_labs
# y_train = y_train_labs.drop(['condition'], axis = 1).values.ravel()
# y_train

x_train_pre = x_train_pre.sort_index()
y_train = y_train.sort_index()
# pd.merge(x_train, y_train, left_index=True, right_index=True) # to merge back

x_train_pre
y_train

# Some exploration 

In [ ]:
# # looking for correlations
# from pandas.plotting import scatter_matrix

# corr_matrix = ap.corr()
# corr_matrix

# attributes = ['Fz', 'Iz', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz'] # failed: attributes = ap.iloc[:,0:10]
# scatter_matrix(ap[attributes], figsize=(12, 8))

# Pipeline 

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer # if joining numerical and categorical pipelines 

In [ ]:
# numerical pipeline

# ap_num = ap.drop(['id', 'condition', 'epoch'], axis = 1)
# num_attribs = list(ap_num)

num_pipeline = Pipeline([ # construct pipeline
        ('imputer', SimpleImputer(strategy="median")),
        ('scaler', StandardScaler()), # consider RobustScaler if there are many outliers
    ])

x_train = num_pipeline.fit_transform(x_train_pre) # apply pipeline

type(x_train) # see
x_train_pre
x_train

In [ ]:
# transform to pandas dataframe
x_train = pd.DataFrame.from_records(x_train) # back to df
x_train.columns = x_train_pre.columns # copy column names

In [ ]:
x_train.shape
y_train.shape

In [ ]:
# save
with open(pkls + 'xy_train.pkl', 'wb') as handle:
    pickle.dump(x_train, handle)
    pickle.dump(y_train, handle)

In [ ]:
# save with quesionnaire
with open(pkls + 'xy_train_q.pkl', 'wb') as handle:
    pickle.dump(x_train, handle)
    pickle.dump(y_train, handle)

### PCA 

In [ ]:
from sklearn.decomposition import PCA, KernelPCA
from sklearn.metrics import mean_squared_error

In [ ]:
pca = PCA(n_components =  0.98) # 2.5 SD = 0.9876, 3 SD = 0.9973
x_train_reduced = pca.fit_transform(x_train)

len(x_train.columns) # all components
len(pca.explained_variance_ratio_) # preserved components/dimensions
sum(pca.explained_variance_ratio_) # preserved variance
len(pca.explained_variance_ratio_)/len(x_train.columns) # % of preserved data

In [ ]:
with open(pkls + 'x_train_reduced.pkl', 'wb') as handle:
    pickle.dump(x_train_reduced, handle)

In [ ]:
cumsum = np.cumsum(pca.explained_variance_ratio_)
plt.close()
plt.figure(figsize=(8, 5))
plt.plot(cumsum)
# x_train.sort_index().cumsum().plot()

### Kernel PCA 

In [ ]:
rbf_pca = KernelPCA(n_components = 2, kernel="rbf", gamma=0.04)

In [ ]:
rbf_pca = KernelPCA(n_components = 2, kernel="rbf", gamma=0.0433, fit_inverse_transform=True)
x_reduced = rbf_pca.fit_transform(x_train)
x_preimage = rbf_pca.inverse_transform(x_reduced)

mean_squared_error(x_train, x_preimage)

In [ ]:
np.linspace(65,85,3, dtype = int)
np.logspace(start = 1, stop = 100, num = 3)
np.array([1,2])
np.power(np.array([1,100]),2)

In [ ]:
param_grid = {'gamma': [1, 0.1, 0.01],
#               'n_components': [65,75,85],
              'kernel': ['rbf']}

def my_scorer(estimator, x_train, y=None):
    x_reduced = estimator.transform(x_train)
    x_preimage = estimator.inverse_transform(x_reduced)
    return mean_squared_error(x_train, x_preimage)

grid_kpca = GridSearchCV(KernelPCA(n_components = 70, fit_inverse_transform=True), 
                         param_grid, scoring=my_scorer, cv = 2)

grid_kpca.fit(x_train)
grid_kpca.best_estimator_

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ("kpca", KernelPCA(n_components=2)),
    ("log_reg", LogisticRegression())
    ])

param_grid = [{
    "kpca__gamma": np.linspace(0.03, 0.05, 10),
    "kpca__kernel": ["rbf", "sigmoid"]
    }]

grid_search = GridSearchCV(clf, param_grid, cv=3)
grid_search.fit(x_train, y_train.values.ravel())

grid_search.best_params_